In [1]:
import linecache
import math
def get_coord(file, num):
    line = linecache.getline(file, num)
    x = float(line[26:38])
    y = float(line[38:46])
    z = float(line[46:54])
    return x, y, z

def distance(x1, y1, z1, x2, y2, z2):
    dx = (x2 - x1)
    dy = (y2 - y1)
    dz = (z2 - z1)
    return dx, dy, dz
    
def translate_PDB(filename, tr_x, tr_y, tr_z, outfile):
    inp = open(filename, 'r')
    out = open(outfile, 'w')
    for line in inp:
        if line.startswith('ATOM'):
            init = line[0:26]
            x = float(line[26:38])
            y = float(line[38:46])
            z = float(line[46:54])
            end = line[54:-1]
            n_x = x + tr_x
            n_y = y + tr_y
            n_z = z + tr_z
            ss = '{}{:>12.3f}{:>8.3f}{:>8.3f}{}{}'.format(init, n_x, n_y, n_z, end, "\n")
            out.writelines(ss)
        else:
            out.writelines(line)

x1, y1, z1 = get_coord('repA_dna30.pdb', 5444)
print(x1, y1, z1)
x2, y2, z2 = get_coord('repA_dna45.pdb', 6305)
print(x2,y2,z2)

dx, dy, dz = distance(x2, y2, z2, x1, y1, z1)
print(dx, dy, dz)
translate_PDB('repA_dna45.pdb', dx, dy, dz, 'translated_45_test.pdb')

-92.153 105.739 119.765
-94.052 31.309 155.215
1.899000000000001 74.43 -35.45


In [89]:
def separate_DNA_Prot(filename, prot_file, DNA_file):
    prot = open(prot_file, 'w')
    dna = open(DNA_file, 'w')
    num_lines = sum(1 for _ in open(filename))
    print(num_lines)
    for i in range(1, 5426):
        line = linecache.getline(filename, i)
        prot.writelines(line)
    for j in range(5426, num_lines+1):
        line = linecache.getline(filename, j)
        dna.writelines(line)   
separate_DNA_Prot('repA_dna30.pdb', 'repA1.pdb', 'dna30.pdb')
separate_DNA_Prot('translated_45_test.pdb', 'repA2.pdb', 'dna45.pdb')

6019
6305


In [90]:
def combine_DNA(file1, file2, outfl):
    num1 = sum(1 for _ in open(file1))
    num2 = sum(1 for _ in open(file2))
    print(num1, num2)
    out = open(outfl, 'w')
    for i in range(1, num1+1):
        line = linecache.getline(file1, i)
        if line.startswith('ATOM'):
            out.writelines(line)
        else:
            print(line)
    atnum1 = int(linecache.getline(file1, num1-1)[4:11])
    resnum1 = int(linecache.getline(file1, num1-1)[22:26])
    for j in range(1, num2+1):
        line = linecache.getline(file2, j)
        if line.startswith('ATOM'):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:22]
            res = int(line[22:26])
            end = line[26:-1]
            n_at = atnum1+j
            n_res = res + resnum1
            ss = '{}{:>7d}{}{:>4d}{}{}'.format(start, n_at, mid, n_res, end, '\n')
            out.writelines(ss)
        else:
            print(ss)
                   
combine_DNA('dna45.pdb', 'dna30.pdb', 'combined_dna.pdb')      

861 594
ATOM   6878  C5'   C D  74     -81.526  34.149 126.449  1.00  0.00           C



In [95]:
def combine_protein(file1, file2, outfl):
    num1 = sum(1 for _ in open(file1))
    num2 = sum(1 for _ in open(file2))
    out = open(outfl, 'w')
    for i in range(1, num1+1):
        line = linecache.getline(file1, i)
        out.writelines(line)
    atnum1 = int(linecache.getline(file1, num1)[4:11])
    for j in range(1, num2+1):
        line = linecache.getline(file2, j)
        if line.startswith(('ATOM')):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:20]
            chain = line[20:22]
            end = line[22:-1]
            n_at = atnum1+j
            if chain==' A':
                n_ch = ' D'
            if chain==' B':
                n_ch = ' E'
            if chain==' C':
                n_ch = ' F'
            ss = '{}{:>7d}{}{:>2s}{}{}'.format(start, n_at, mid, n_ch, end, '\n')
            out.writelines(ss)
        if line.startswith(('TER')):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:20]
            chain = line[20:22]
            end = line[22:-1]
            n_at = atnum1+j
            if chain==' A':
                n_ch = ' D'
            if chain==' B':
                n_ch = ' E'
            if chain==' C':
                n_ch = ' F'
            ss = '{}{:>7d}{}{:>2s}{}{}'.format(start, n_at, mid, n_ch, end, '\n')
            out.writelines(ss)
combine_protein('repA1.pdb', 'repA2.pdb', 'two_RPA.pdb') 

In [100]:
def combine_protein_DNA(file1, file2, outfl):
    num1 = sum(1 for _ in open(file1))
    num2 = sum(1 for _ in open(file2))
    out = open(outfl, 'w')
    for i in range(1, num1+1):
        line = linecache.getline(file1, i)
        out.writelines(line)
    atnum1 = int(linecache.getline(file1, num1)[4:11])

    for j in range(1, num2+1):
        line = linecache.getline(file2, j)
        if line.startswith(('ATOM')):
            start = line[0:4]
            at = int(line[4:11])
            mid = line[11:20]
            chain = line[20:22]
            end = line[22:-1]
            n_at = atnum1+j
            if chain==' D':
                n_ch = ' G'
            ss = '{}{:>7d}{}{:>2s}{}{}'.format(start, n_at, mid, n_ch, end, '\n')
            out.writelines(ss)
        else:
            out.writelines(line)
combine_protein_DNA('two_RPA.pdb', 'combined_dna.pdb', 'RPA_DNA.pdb') 

In [150]:
filename= 'RPA_DNA.dat'
string1 = '7 chains.'
init_line = next((i+1 for i, line in enumerate(open(filename)) if string1 in line), None)
print(init_line)
end_line = sum(1 for _ in open(filename))
print(end_line)

trp_list = []
tyr_list = []
count = 0
for i in range(init_line+8, end_line):
    line = linecache.getline(filename, i)
    res = line[12:16].strip()
    res_num = line[0:5].strip()
    type = line[9:12].strip()
    if res=='HIS' and type=='CB':
        trp_list.append(res_num)
    if res==' TYR':
        tyr_list.append(res_num)
print(trp_list)  


4179425
4182332
['1392', '1509', '1605', '1771', '1797', '1809', '1813', '1882', '1962', '2273', '2295', '2443', '2547', '2577']
